# 1. 라이브러리 및 API key 설정

In [3]:
import os
import torch
import numpy as np
import datasets
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import roc_auc_score
from openai import OpenAI
from serpapi import GoogleSearch 
from dotenv import load_dotenv
import pandas as pd
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_anthropic import ChatAnthropic, AnthropicLLM
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import json
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()


True

In [47]:
# OpenAI 및 SerpAPI 키 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERP_API_KEY = os.getenv("SERPAPI_API_KEY")
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=OPENAI_API_KEY)

# FAISS 인덱스 저장 경로
FAISS_INDEX_PATH = Path('local/news-please/faiss_index')

# 2. 요약


In [46]:
import os
import json
import wikipedia
import nltk
import pandas as pd

# LangChain 관련
from langchain.llms import OpenAI  # 예시로 OpenAI 사용
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 의학과학개요 검토 LLM
llm1 = ChatOpenAI(temperature=1,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-4o-mini',  # 모델명
                )

llm2 = ChatAnthropic(model='claude-3-5-haiku-20241022',max_tokens=8192)

########################################
# 1. LangChain을 이용한 키워드 추출 함수
########################################
import json

def extract_sentences(n, tf, e) -> dict:
    """
    Extracts key sentences and explanations from the given news and evidence.
    """

    prompt = PromptTemplate(
        input_variables=["news", "news_veracity", "evidence"],
        template=(
            """
            News: {news}
            News Veracity (Hidden from Student): {news_veracity}  # 1 for True, 0 for False
            Evidence: {evidence}

            Task: You are a teacher summarizing information for a student. Your goal is to help the student determine whether the given news is true or false based on the provided evidence. The news veracity is hidden from the student.

            Carefully read all the sentences in the given evidence. Identify and extract all sentences that are crucial for determining the veracity of the news.

            Additionally, summarize the evidence into 30 sentences that are easy for the student to read. Ensure that the key sentences are included in the summary, but also mix in less important sentences to encourage the student to develop judgment skills.

            - Do NOT add any new information beyond the original evidence.
            - Return ONLY a JSON object with no additional text.

            If none of the sentences in the evidence are relevant for determining the veracity of the news, return an empty list for "key sentences".

            STRICTLY return the output in JSON format, without any additional text with the following structure:

            <output structure>
            {{
                "key sentences": ["sentence1", "sentence2", "sentence3"],
                "key sentences explanation": [
                    "Sentence1 supports the news because it provides factual evidence aligned with the claim.",
                    "Sentence2 contradicts the news by presenting counter-evidence that disputes the claim.",
                    "Sentence3 supports the news as it comes from an authoritative source verifying the information."
                ],
                "summarized evidence sentences": ["sentence1", "sentence2", ..., "sentence30"]
            }}
            </output structure>
            """
        )
    )

    chain = prompt | llm1 | StrOutputParser()

    try:
        response_text = chain.invoke({"news": n, "news_veracity": tf, "evidence": e}).strip()
        
        # Ensure the response starts and ends correctly as JSON
        if not response_text.startswith("{") or not response_text.endswith("}"):
            raise ValueError("Invalid JSON response from LLM")

        response_json = json.loads(response_text)
        return response_json
    
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        print("Raw LLM Response:", response_text)
        return {"error": "Invalid JSON format returned from LLM"}

In [52]:
df = data.iloc[0:100,:]
df

,news,extracted_entities,wiki_summary,wiki_full,label,title,news_merged
0,u.s. senate republican leader mitch mcconnell ...,"Mitch McConnell, Myanmar, Aung San Suu Kyi, Jo...",Mitch McConnell: Addison Mitchell McConnell II...,Mitch McConnell: Addison Mitchell McConnell II...,1,senate leader opposes lecturing myanmar leader...,title : senate leader opposes lecturing myanma...
1,four u.s. senators - two democrats and two rep...,"senators charles grassley, senator patrick lea...",senators charles grassley: Charles Ernest Gras...,senators charles grassley: Charles Ernest Gras...,1,senators introduce bill aimed at getting gener...,title : senators introduce bill aimed at getti...
2,note little johnny might want consider use dia...,"little johnny, isis, fort riley military base,...",little johnny: Little Johnny jokes are about a...,little johnny: Little Johnny jokes are about a...,1,NaN,content : note little johnny might want consid...
3,president obama is clearly in good spirits tha...,"president obama, republican party, trump presi...",president obama: Barack Hussein Obama II (born...,president obama: Barack Hussein Obama II (born...,0,president obama gets his troll game on by than...,title : president obama gets his troll game on...
4,turkey said on monday its former economy minis...,"Zafer Caglayan, Bekir Bozdag, FETO, Tayyip Erd...",Zafer Caglayan: Mehmet Zafer Çağlayan (born 10...,Zafer Caglayan: Mehmet Zafer Çağlayan (born 10...,1,turkey says u.s. indictment of former minister...,title : turkey says u.s. indictment of former ...
...,...,...,...,...,...,...,...
95,a suicide car bomber blew himself up at a secu...,"Islamic State, Houthi rebels, Yemen, Al Qaeda ...","Islamic State: The Islamic State (IS), also kn...","Islamic State: The Islamic State (IS), also kn...",1,car bomber kills at least 15 in yemeni port ad...,title : car bomber kills at least 15 in yemeni...
96,london british prime minister theresa may rais...,"Theresa May, Benjamin Netanyahu, London, Israe...","Theresa May: Theresa Mary May, Baroness May of...","Theresa May: Theresa Mary May, Baroness May of...",0,NaN,content : london british prime minister theres...
97,the obama administration s cleansing of the is...,"Omar Mateen, Abu Bakr al-Baghdadi, Paul D. Rya...",Omar Mateen: Omar Mir Seddique Mateen (Pashto:...,Omar Mateen: Omar Mir Seddique Mateen (Pashto:...,0,two words obamas doj has been hiding from orla...,title : two words obamas doj has been hiding f...
98,the intimidation tactics of mueller s fbi are ...,"Robert Mueller, Paul Manafort, Andrew Weissman...",Robert Mueller: Robert Swan Mueller III (; bor...,Robert Mueller: Robert Swan Mueller III (; bor...,0,muellers fbi raid of paul manafort and wife pr...,title : muellers fbi raid of paul manafort and...


In [57]:
import pandas as pd
import json

# 데이터 로드 및 초기화
data = pd.read_csv("/Users/yoonjincho/Desktop/윤진 2025/YAI/YAI_personal_archive/NLP_ToyProject/merged_data_0302_2차.csv").drop(columns=['Unnamed: 0'])
output_path = "output_0302_0-100.csv"  # 결과 CSV 파일명
df = data.iloc[0:100, :]

# 결과 컬럼 추가 (초기화)
df["key sentences"] = None
df["key sentences explanation"] = None
df["summarized evidence sentences"] = None

# 각 행 처리
for idx, row in df.iterrows():
    news_text = row["news_merged"]
    news_veracity = row["label"]  # 라벨 컬럼명 확인 필요
    evidence = row["wiki_summary"]

    try:
        # 뉴스에서 키워드(엔티티) 추출
        response = extract_sentences(news_text, news_veracity, evidence)

        # 응답이 유효한 JSON 객체인지 확인
        if isinstance(response, dict) and "key sentences" in response and "key sentences explanation" in response and "summarized evidence sentences" in response:
            df.at[idx, "key sentences"] = json.dumps(response["key sentences"], ensure_ascii=False)
            df.at[idx, "key sentences explanation"] = json.dumps(response["key sentences explanation"], ensure_ascii=False)
            df.at[idx, "summarized evidence sentences"] = json.dumps(response["summarized evidence sentences"], ensure_ascii=False)
        else:
            print(f"⚠️ Warning: Unexpected response format at index {idx}")
    
    except Exception as e:
        print(f"❌ Error at index {idx}: {e}")

    # 10의 배수일 때마다 파일 저장
    if (idx + 1) % 10 == 0:
        df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"✅ Progress Saved: {output_path} at index {idx}")

# 최종 저장
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"🎉 Processing complete. Final output saved to '{output_path}'")

/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_88251/2834649438.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["key sentences"] = None
/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_88251/2834649438.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["key sentences explanation"] = None
/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_88251/2834649438.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

❌ Error at index 1: Invalid JSON response from LLM
❌ Error at index 4: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 9
❌ Error at index 18: Invalid JSON response from LLM
❌ Error at index 19: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 19
✅ Progress Saved: output_0302_0-100.csv at index 29
✅ Progress Saved: output_0302_0-100.csv at index 39
❌ Error at index 47: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 49
✅ Progress Saved: output_0302_0-100.csv at index 59
✅ Progress Saved: output_0302_0-100.csv at index 69
✅ Progress Saved: output_0302_0-100.csv at index 79
❌ Error at index 83: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 89
❌ Error at index 99: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 99
🎉 Processing complete. Final output saved to 'output_0302_0-100.csv'


In [56]:
df

,news,extracted_entities,wiki_summary,wiki_full,label,title,news_merged,key sentences,key sentences explanation,summarized evidence sentences
0,u.s. senate republican leader mitch mcconnell ...,"Mitch McConnell, Myanmar, Aung San Suu Kyi, Jo...",Mitch McConnell: Addison Mitchell McConnell II...,Mitch McConnell: Addison Mitchell McConnell II...,1,senate leader opposes lecturing myanmar leader...,title : senate leader opposes lecturing myanma...,"[""U.S. Senate Republican leader Mitch McConnel...","[""This sentence supports the news because it d...","[""Mitch McConnell is the senior United States ..."


In [25]:
summary

'Myanmar, officially the Republic of the Union of Myanmar and also rendered as Burma (the official English form until 1989), is a country in northwest Southeast Asia. It is the largest country by area in Mainland Southeast Asia and has a population of about 55 million. It is bordered by India and Bangladesh to its northwest, China to its northeast, Laos and Thailand to its east and southeast, and the Andaman Sea and the Bay of Bengal to its south and southwest. The country\'s capital city is Naypyidaw, and its largest city is Yangon (formerly Rangoon).\nEarly civilisations in the area included the Tibeto-Burman-speaking Pyu city-states in Upper Myanmar and the Mon kingdoms in Lower Myanmar. In the 9th century, the Bamar people entered the upper Irrawaddy valley, and following the establishment of the Pagan Kingdom in the 1050s, the Burmese language and culture and Theravada Buddhism slowly became dominant in the country. The Pagan Kingdom fell to Mongol invasions, and several warring s